In [1]:
import numpy as np
import numpy.random as rn
from numba import jit, njit, prange
import matplotlib.pyplot as plt

In [2]:
# @njit
# @jit(nopython=True, parallel=True)
def random_initial_state_generator(L, concentration): # Checked OK
    state = np.zeros((L, L, L))
    index_list = []


    for i in prange(L):
        for j in prange(L):
            for k in prange(L):
                if rn.rand() <= 0.5:
                    state[i, j, k] = -1
                else:
                    state[i, j, k] = 1

    for i in prange(L):
        for j in prange(L):
            for k in prange(L):
                index_list.append([i, j, k])

    rn.shuffle(index_list)

    for i in prange(int(round(((1 - concentration) * L * L * L), 5))):
        coordinate = index_list.pop(0)
        state[coordinate[0], coordinate[1], coordinate[2]] = 0
    
    return state


@jit(nopython=True, parallel=True)
def initial_state_test(L, state):
    test_conc = 0

    for i in prange(L):
        for j in prange(L):
            for k in prange(L):
                if state[i, j, k] == 0:
                    test_conc += 1
    
    return (test_conc / (L * L * L))


@jit(nopython=True, parallel=True)
def calculate_magnetization_per_spin(state, L): # Checked OK
  magnetization = 0

  for i in prange(L):
    for j in prange(L):
      for k in prange(L):
        magnetization += state[i, j, k]

  return magnetization / (L**3)


@jit(nopython=True, parallel=True)
def calculate_energy_per_spin(state, L, J, B=0): # Checked OK
    energy_J = 0
    energy_B = 0

    for i in prange(L):
        for j in prange(L):
            for k in prange(L):
                i_prev = (i - 1) % L
                i_next = (i + 1) % L
                j_prev = (j - 1) % L
                j_next = (j + 1) % L
                k_prev = (k - 1) % L
                k_next = (k + 1) % L
                
                energy_J += -J * state[i, j, k] * (state[i_prev, j, k] + state[i_next, j, k] + state[i, j_prev, k] + state[i, j_next, k] + state[i, j, k_prev] + state[i, j, k_next])
                energy_B += -B * state[i, j, k]

    return (energy_J + energy_B) / (2 * L**3)


@jit(nopython=True, parallel=True)
def next_state_generator(state, L):
    site = np.array([rn.randint(0, L - 1), rn.randint(0, L - 1), rn.randint(0, L - 1)])
    site_spin = state[site[0], site[1], site[2]]

    if site_spin == 1:
        state[site[0], site[1], site[2]] = -1
    elif site_spin == -1:
        state[site[0], site[1], site[2]] = 1

    return state, site


@jit(nopython=True, parallel=True)
def change_in_energy(current_state, next_state, site, L, J, B=0):
    i = site[0]
    j = site[1]
    k = site[2]

    i_prev = (i - 1) % L
    i_next = (i + 1) % L
    j_prev = (j - 1) % L
    j_next = (j + 1) % L
    k_prev = (k - 1) % L
    k_next = (k + 1) % L

    current_state_site_energy = -J * current_state[i, j, k] * (current_state[i_prev, j, k] + current_state[i_next, j, k] + current_state[i, j_prev, k] + current_state[i, j_next, k] + current_state[i, j, k_prev] + current_state[i, j, k_next])

    next_state_site_energy = -J * next_state[i, j, k] * (next_state[i_prev, j, k] + next_state[i_next, j, k] + next_state[i, j_prev, k] + next_state[i, j_next, k] + next_state[i, j, k_prev] + next_state[i, j, k_next])

    return next_state_site_energy - current_state_site_energy


@jit(nopython=True, parallel=True)
def central_difference_derivative(x_array, y_array):
    N = len(x_array)
    cnt_der = np.zeros((N - 2))

    for i in prange(1, N-1):
        cnt_der[i - 1] = (y_array[i + 1] - y_array[i - 1]) / (x_array [i + 1] - x_array[i - 1])
    
    return cnt_der


In [ ]:
L = 5
concentration = 1
J = 1

In [ ]:
L = 4
concentration = 1
J = 1

random_initial_state = random_initial_state_generator(L, concentration)
# print(random_initial_state)
print(initial_state_test(L, random_initial_state))
print(calculate_magnetization_per_spin(random_initial_state, L))
# print(calculate_magnetization_per_spin(random_initial_state, L)*(30**3))
print(calculate_energy_per_spin(random_initial_state, L, J, B=0))

state, site = next_state_generator(random_initial_state, L)
print(site)
print(state)

In [51]:
# @jit(nopython=True, parallel=True)
@njit
def ising_mc_thermalization(random_state, T, L, therm_steps, J):
    state = random_state.copy()

    for l in prange(therm_steps):
        for s in prange(L * L * L):
            next_state = state.copy()

            i = rn.randint(L)
            j = rn.randint(L)
            k = rn.randint(L)

            if next_state[i, j , k] == 0:
                break

            next_state[i, j , k] = -1 * state[i, j, k]
            energy_change = change_in_energy(state, next_state, [i, j, k], L, J)

            P_acc = np.exp(-1 * energy_change / T)

            if energy_change < 0:
                state = next_state.copy()
            
            elif rn.rand() <= P_acc:
                state = next_state.copy()

    return state



@njit
def ising_mc_simulation(thermalized_state, T, L, mc_steps, J, skip_step = 10):
    state = thermalized_state.copy()

    energy_per_spin = 0
    magnetization_per_spin = 0

    for l in prange(mc_steps):
        for s in prange(L * L * L):
            next_state = state.copy()

            i = rn.randint(L)
            j = rn.randint(L)
            k = rn.randint(L)

            if next_state[i, j , k] == 0:
                break

            next_state[i, j , k] = -1 * state[i, j, k]
            energy_change = change_in_energy(state, next_state, [i, j, k], L, J)

            P_acc = np.exp(-1 * energy_change / T)

            if energy_change < 0:
                state = next_state.copy()
            
            elif rn.rand() <= P_acc:
                state = next_state.copy()
    
        if (l + 1) % skip_step == 0:
            energy_per_spin += calculate_energy_per_spin(state, L, J, B=0)
            magnetization_per_spin += calculate_magnetization_per_spin(state, L)
    
    return ((energy_per_spin * skip_step) / mc_steps), ((magnetization_per_spin * skip_step) / mc_steps)


In [ ]:
T = 1
concentration = 1
J = 1
L = 30

therm_steps = 1000
mc_steps = 1000
skip_step = 10


random_state = random_initial_state_generator(L, concentration)

thermalized_state = ising_mc_thermalization(random_state, T, L, therm_steps, J)

av_energy_per_spin, av_magnetization_per_spin = ising_mc_simulation(thermalized_state, T, L, mc_steps, J, skip_step = skip_step)

print(av_energy_per_spin, '\n', av_magnetization_per_spin)

# print("Checking")
# for i in range(50):
#     print(i, "--------",ising_mc_simulation(thermalized_state, T, L, mc_steps, J, skip_step = skip_step))

In [168]:
T_0 = 6
m = 0
T_max = 0.1
T_step_size = -0.1
L = 10
J = 1
concentration = 1
therm_steps = 1000
mc_steps = 1000


temp_array = np.arange(T_0, T_max + T_step_size, T_step_size)
temp_array = np.round(temp_array, 3)


print(temp_array)
print(len(temp_array))


energy_by_T_array = np.zeros(len(temp_array))
magnetization_by_T_array = np.zeros(len(temp_array))
susceptibility_by_T_array = np.zeros(len(temp_array))


for T in temp_array:
    print(T)
    state = random_initial_state_generator(L, concentration)

[6.  5.9 5.8 5.7 5.6 5.5 5.4 5.3 5.2 5.1 5.  4.9 4.8 4.7 4.6 4.5 4.4 4.3
 4.2 4.1 4.  3.9 3.8 3.7 3.6 3.5 3.4 3.3 3.2 3.1 3.  2.9 2.8 2.7 2.6 2.5
 2.4 2.3 2.2 2.1 2.  1.9 1.8 1.7 1.6 1.5 1.4 1.3 1.2 1.1 1.  0.9 0.8 0.7
 0.6 0.5 0.4 0.3 0.2 0.1]
60
